In [11]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np

This class was originally seen on https://github.com/pytorch/pytorch/issues/1333 and I am just trying to understand it so that I will then be able to do my own

In [15]:
class CausalConv1d(nn.Conv1d):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 stride=1,
                 dilation=1,
                 groups=1,
                 bias=True):
        super(CausalConv1d, self).__init__(
            in_channels,
            out_channels,
            kernel_size,
            stride=stride,
            padding=0,
            dilation=dilation,
            groups=groups,
            bias=bias)

        self.left_padding = dilation * (kernel_size - 1)

    def forward(self, input):
        x = F.pad(input.unsqueeze(2), (self.left_padding, 0, 0, 0)).squeeze(2)

        return super(CausalConv1d, self).forward(x)

In [45]:
causal = CausalConv1d(4, 1, 2)
print(causal)

CausalConv1d (4, 1, kernel_size=(2,), stride=(1,))


This gives a very large set of parameters in a weird large array, which I dont yet understand

In [46]:
params = list(causal.parameters())
print(params)

[Parameter containing:
(0 ,.,.) = 
  0.2245 -0.0339
 -0.1332 -0.0178
  0.2899 -0.2845
 -0.2659  0.0192
[torch.FloatTensor of size 1x4x2]
, Parameter containing:
1.00000e-02 *
 -6.3724
[torch.FloatTensor of size 1]
]


In [47]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(causal.parameters(), lr=0.001, momentum=0.9)

Create the input and target vectors as numpy arrays, then turn them to torch FloatTensor

In [48]:
x = np.array([[1,5,2,3]]) # by placing additional brackets around the array, it goes from size 4 to 1*4
input = Variable(torch.from_numpy(x).type(torch.FloatTensor)) # turn a numpy array to a torch tensor and specify the type
print(input)

y = np.array([[0,2,10,4]])
target = Variable(torch.from_numpy(y).type(torch.FloatTensor))
print(target)

Variable containing:
 1  5  2  3
[torch.FloatTensor of size 1x4]

Variable containing:
  0   2  10   4
[torch.FloatTensor of size 1x4]



In [49]:
out = causal(input)
print(out)

Variable containing:
(0 ,.,.) = 
 -0.6981
[torch.FloatTensor of size 1x1x1]



In [34]:
# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = causal(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()

ValueError: Expected 2 or 4 dimensions (got 3)

In [ ]:
causal.zero_grad()
out.backward(torch.randn(1, 10))

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(causal.parameters(), lr=0.001, momentum=0.9)